In [1]:
#from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import load_model
from music21 import *
import music21
import os
import glob
import re
import numpy as np
import math
import pickle

In [2]:
#global path
HMM_root_folder='../modules'
HMM_matrix_folder='../results/HMM'
LSTM_Key_prediction_model_root='.'

import sys
sys.path.append(HMM_root_folder)
sys.path.append("../RL modules/")
import HMM
from env_noOctave import SegmentationEnv

In [3]:
#Segmentation

def rlsegment(piece):
    """
    rlsegment: returns a list of offset
    Input:
        string piece -- path string to the file location
    Returns:
        list of tuples (a,b): a is the start offset and b is the end offset of a segment
    """
    env = SegmentationEnv([piece])
    model = load_model("../results/dqn_normalized_2")
    offsets = []
    obs = env.reset(0)
    final_offset = env.offset[0][-1]
    while True:
        obs = obs.reshape((1, 12 * 2 + 1))
        action = np.argmax(model.predict(obs))
        if action == 1:
            offsets.append(env.current_noteoffset)
        obs, reward, done, info = env.step(action)
        if done:
            break
    toreturn = [(0, offsets[0])]
    for i in range(1, len(offsets)):
        toreturn.append((offsets[i - 1], offsets[i]))
    toreturn.append((offsets[-1], final_offset))
    return toreturn

In [4]:
#Key-prediction
class Score:
    def __init__(self):
        self.beat_list=[Beat()]
        
    #extract info from note and add to corrsponding beat
    def add_note(self,note):
        length=note.quarterLength
        start=cal_offset(note)
        end=start+length
        rounded_floor_start=math.floor(start)
        #loop until the note played to its end
        while start<end-0.000000000001:
            if len(self.beat_list)-1<rounded_floor_start:
                new_beat=rounded_floor_start-(len(self.beat_list)-1)
                #the input note maybe is a chord -> recurse all pitch inside
                for _ in range(new_beat):
                    self.beat_list.append(Beat())
            self.beat_list[rounded_floor_start].add_note(note,min(rounded_floor_start+1-start,end-start))
            start+=min(rounded_floor_start+1-start,end-start)
            rounded_floor_start=int(start)
            
    #add key to the first occurence of beat
    def add_key(self,note):
        assert(note.lyric is not None and '(' in note.lyric)
        key_change_beat=cal_offset(note)
        rounded_floor_key_change_beat=math.floor(key_change_beat)
        self.beat_list[rounded_floor_key_change_beat].add_key(note.lyric.split('(')[0])
        
    #onyl call once
    def infer_key(self):
        first_key_in_num=None
        first_key_full=None
        first_key_major=None
        #backtrack
        for e in self.beat_list:
            if e.key_full is not None:
                first_key_full=e.key_full
                first_key_in_num=e.key_in_num
                first_key_major=e.major
                break
        #bring forward
        for e in self.beat_list:
            if e.key_full is None:
                e.key_full=first_key_full
                e.key_in_num=first_key_in_num
                e.major=first_key_major
            else:
                first_key_full=e.key_full
                first_key_in_num=e.key_in_num
                first_key_major=e.major
                
class Beat:
    key_mapping={
        'C':0,
        'D':2,
        'E':4,
        'F':5,
        'G':7,
        'A':9,
        'B':11
    }
    def __init__(self):
        self.notes = np.zeros((12,7))  #from C1 to C7
        self.total_duration = np.zeros((12,7))
        self.notes_occurences_count= np.zeros((12,7))
        self.key_full=None
        self.major=None
        self.key_in_num=None
    def key2num(self,k):  
        k=k.upper()
        num=self.key_mapping[k[0]]
        modifier=len(k)
        if modifier==1:
            return num
        elif k[1]=='#':
            return (num+(modifier-1))%12
        elif k[1]=='B' or k[1]=='-' or k[1]=='♭':
            return (num-(modifier-1))%12
        elif k[1]=='X':
            return (num+(modifier-1)*2)%12
    def add_note(self,note,duration):
        assert(duration<=1)
        pitches=note.pitches
        for pitch in pitches:
            pitch_idx=self.key2num(pitch.nameWithOctave[:-1])
            octave=int(pitch.nameWithOctave[-1])-1
            if octave<0:
                octave=0
            elif octave>6:
                octave=6
            self.notes[pitch_idx,octave]=1
            self.total_duration[pitch_idx,octave]+=duration
            self.notes_occurences_count[pitch_idx,octave]+=1
            
    def add_key(self,k):
        self.major = 'M' in k
        self.key_full=k
        k=k[:-1]
        self.key_in_num=self.key2num(k)
        
#turn the data(ordered in score) to tranable format(ordered in segments)
def trainable(X):
    retX=[ b for x in X for b in x ]
    retX=np.array(retX)
    return retX

def key_prediction(piece):
    #LSTM - keyprediction
    key_prediction_model = tf.keras.models.load_model(LSTM_Key_prediction_model_root+'/key_prediction.h5')
    #load data beat by beat
    predict_score=Score()
    print('train',piece)
    chords = []
    notes = []
    c = converter.parse(piece)
    post = c.flat

    #extract note
    for note in post.notes:
        predict_score.add_note(note)
        if note.lyric is not None and '(' in note.lyric:
            predict_score.add_key(note)
    #predict_score.infer_key()
    predict_score=[predict_score]
    
    #turns each beat to a vector
    weight=[1.35,1.25,0.9,0.8,0.9,1,1.2]
    trainX=[]
    for e in predict_score:
        tempX=[]
        for beat in e.beat_list:
            value=beat.total_duration*beat.notes_occurences_count
            if np.sum(value)!=0:
                value/=np.sum(value)
                value*=weight
                value=value.sum(axis=1)
                #value=value.reshape((-1))
                value/=value.sum()
            else:
                value=np.zeros((12))

            assert(len(value)==12)
            tempX.append(value)
        trainX.append(tempX)

        
    #generate segments, ordered in score
    look_forward=4
    look_after=4
    dataX=[]
    for idx_p,piece in enumerate(trainX):
        piece_notesX=[]
        for idx_b,beat in enumerate(piece):
            tempX=[]
            tempfront=[]
            tempend=[]
            for i in reversed(range(1,look_forward+1)):
                if(idx_b-i)<0:
                    tempfront.append(np.zeros(12))
                else:
                    tempfront.append(np.array(piece[idx_b-i]))
            tempfront=np.array(tempfront)
            tempfront=np.sum(tempfront,axis=0)

            tempX.append(tempfront)
            tempX.append(piece[idx_b])

            for i in range(1,look_after+1):
                if(idx_b+i)>len(piece)-1:
                    tempend.append(np.zeros(12))
                else:
                    tempend.append(np.array(piece[idx_b+i]))

            tempend=np.sum(tempend,axis=0) 
            tempX.append(tempend)

            piece_notesX.append(tempX)
        dataX.append(piece_notesX)
        
    #reformat the data
    processedX=trainable(dataX)
    return key_prediction_model.predict(processedX)


In [5]:
#HMM model
#HMM model(chord prediction)

h_states=[#Minor
      'MinorI', 'MinorI+',
      'MinorbII', 'MinorII',
      'MinorIII',
      'MinorIV', 'MinorIV+',
      'MinorV', 'MinorV+',
      'MinorVI', 'MinorGerVI', 'MinorFreVI', 'MinorItaVI',
      'MinorVII', 'MinorDimVII',
      #Major
      'MajorI',
      'MajorbII','MajorII',
      'MajorIII',
      'MajorIV',
      'MajorV',
      'MajorbVI','MajorGerVI','MajorFreVI','MajorItaVI','MajorVI',
      'MajorVII'
]

def prepareHMM(piece,offsets):
    weight=[1.35,1.25,0.9,0.8,0.9,1,1.2]
    c = converter.parse(piece)
    all_notes=[]
    for el in c.recurse().notes:
        if el.lyric is not None:
            el.lyric=el.lyric.replace('♭','b')
        all_notes.append([el.lyric, el, cal_offset(el),el.duration.linked])    
    #sort by first occurence
    b=sorted(all_notes,key=lambda x: (x[-2],x[0] if x[0] is not None else "ZZZZZZZZZZZZZZZ"))
    
    HMM_notes=[]
    trace_idx=0
    for segment in offsets:
        data_note={}
        while b[trace_idx][2]<segment[1]:
            for pitch in b[trace_idx][1].pitches:
                if pitch.name in data_note:
                    data_note[pitch.name]+=b[trace_idx][1].quarterLength*weight[int(pitch.nameWithOctave[-1])-1]
                    data_note[pitch.name]*=1.2 #reward for occurence
                else:
                    data_note[pitch.name]=b[trace_idx][1].quarterLength*weight[int(pitch.nameWithOctave[-1])-1]        
            trace_idx+=1
        #
        s=0
        for k,v in data_note.items():
            s+=v
        for k,v in data_note.items():
            data_note[k]=v/s
        HMM_notes.append(data_note)
   
    return HMM_notes

def prepareHMM_key(predicted_key,predicted_major,offsets):
    chords=['C','Db','D','Eb','E','F','F#','G','Ab','A','Bb','B']
    M=['Minor','Major']
    key_name_score=[]
    trace_id=0
    for segment in offsets:
        bag=[]
        while trace_id<segment[1]:
            if (predicted_major[trace_id][0]>=0.5)*1==0:
                name=chords[np.argmax(predicted_key[trace_id])].lower()+M[(predicted_major[trace_id][0]>=0.5)*1]
            else:
                name=chords[np.argmax(predicted_key[trace_id])]+M[(predicted_major[trace_id][0]>=0.5)*1]
            bag.append(name)
            trace_id+=1
        if len(bag)==0:
            #follow previous pred
            key_name_score.append(key_name_score[-1])
        else:
            #majority vote from bag
            key_name_score.append(max(set(bag), key = bag.count))
    return key_name_score
def HMM_predict(note,key):
    h_states=[#Minor
      'MinorI', 'MinorI+',
      'MinorbII', 'MinorII',
      'MinorIII',
      'MinorIV', 'MinorIV+',
      'MinorV', 'MinorV+',
      'MinorVI', 'MinorGerVI', 'MinorFreVI', 'MinorItaVI',
      'MinorVII', 'MinorDimVII',
      #Major
      'MajorI',
      'MajorbII','MajorII',
      'MajorIII',
      'MajorIV',
      'MajorV',
      'MajorbVI','MajorGerVI','MajorFreVI','MajorItaVI','MajorVI',
      'MajorVII'
]
    with open(HMM_matrix_folder+'/initial_matrix.pickle', 'rb') as f:
        initial_matrix = pickle.load(f)
    with open(HMM_matrix_folder+'/transition_matrix.pickle', 'rb') as d:
        transition_matrix = pickle.load(d)
    with open(HMM_matrix_folder+'/emssion_matrix.pickle', 'rb') as e:
        emssion_matrix = pickle.load(e)
    with open(HMM_matrix_folder+'/note_probit.pickle', 'rb') as g:
        note_probit = pickle.load(g)
    hmm_model=HMM.HMM(len(h_states),2,h_states,["outside chord","inside chord"])
    hmm_model.initial_matrix=initial_matrix
    hmm_model.transition_matrix=transition_matrix
    hmm_model.emssion_matrix=emssion_matrix
    hmm_model.note_probit=note_probit
    prediction=hmm_model.predict(note,key)
    return prediction

In [6]:
#global functions
def cal_offset(e):
    if e is None:
        return 0
    return e.offset+cal_offset(e.activeSite)

In [7]:
# get environment
env = environment.Environment()

env['musescoreDirectPNGPath'] = 'C:\Program Files\MuseScore 3\\bin\MuseScore3.exe'
env['musicxmlPath'] = "C:\Program Files\MuseScore 3"
# check the path
print('Environment settings:')
print('musicXML:  ', env['musicxmlPath'])
print('musescore: ', env['musescoreDirectPNGPath'])

# set path if necessary
# env['musicxmlPath'] = 'path/to/your/musicXmlApplication'


Environment settings:
musicXML:   C:\Program Files\MuseScore 3
musescore:  C:\Program Files\MuseScore 3\bin\MuseScore3.exe


In [8]:
#main
#input 
def total_inference(input_score,out):
    offsets=rlsegment(input_score)
    if offsets[-1][0] == offsets[-1][1]:
        del offsets[-1]
    print(offsets)
    predicted_key,predicted_major=key_prediction(input_score)
    HMM_notes=prepareHMM(input_score,offsets)
    HMM_key=prepareHMM_key(predicted_key,predicted_major,offsets)
    key_and_chord_prediction=HMM_predict(HMM_notes,HMM_key)
    prediction_output = []
    for idx,prediction in enumerate(key_and_chord_prediction):
        prediction_output.append(HMM_key[idx][:-5]+h_states[prediction])
    c = converter.parse(input_score)
    all_notes=[]
    for el in c.recurse().notes:
        if el.lyric is not None:
            el.lyric=el.lyric.replace('♭','b')
        all_notes.append([el.lyric, el, cal_offset(el),el.duration.linked])    
    #sort by first occurence
    b=sorted(all_notes,key=lambda x: (x[-2],x[0] if x[0] is not None else "ZZZZZZZZZZZZZZZ"))
    trace_idx=0
    for idx,segment in enumerate(offsets):
        while b[trace_idx][2]<segment[0] or not b[3]:
            trace_idx+=1
        b[trace_idx][1].addLyric(prediction_output[idx])
    c.write('musicxml',out)

In [11]:
total_inference("mozart.mxl","output.musicxml")   
        

mozart.mxl
Total number of pieces 1
[(0, 4.0), (4.0, 6.0), (6.0, 11.0), (11.0, 13.0), (13.0, 16.0), (16.0, 20.0), (20.0, 22.0), (22.0, 24.0), (24.0, 25.0), (25.0, 28.0), (28.0, 30.0), (30.0, 32.0), (32.0, 41.0), (41.0, 44.0), (44.0, 73.0), (73.0, 78.0), (78.0, 82.0), (82.0, 84.0), (84.0, 93.0), (93.0, 96.0), (96.0, 99.0), (99.0, 103.0), (103.0, 104.0), (104.0, 107.0), (107.0, 108.0), (108.0, 119.0), (119.0, 121.0), (121.0, 126.0), (126.0, 131.0), (131.0, 132.0), (132.0, 140.0), (140.0, 144.0), (144.0, 147.0), (147.0, 150.0), (150.0, 153.0), (153.0, 158.0), (158.0, 160.0), (160.0, 162.0), (162.0, 163.0), (163.0, 167.0), (167.0, 170.0), (170.0, 176.0), (176.0, 180.0), (180.0, 183.0), (183.0, 189.0), (189.0, 192.0), (192.0, 199.0), (199.0, 202.0), (202.0, 204.0), (204.0, 207.0), (207.0, 208.0), (208.0, 211.0), (211.0, 212.0), (212.0, 221.0), (221.0, 224.0), (224.0, 233.0), (233.0, 236.0), (236.0, 238.0), (238.0, 242.0), (242.0, 244.0), (244.0, 246.0), (246.0, 251.0), (251.0, 256.0), (256.

In [10]:
c = converter.parse("mozart.mxl")
c.show()

TypeError: argument of type 'WindowsPath' is not iterable

In [ ]:
c = converter.parse("output.musicxml")
c.show()